In [4]:
import numpy as np
import math
import random
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.quantum_info import Statevector, DensityMatrix, state_fidelity
from scipy.linalg import eigvalsh

# -------- métricas --------
def von_neumann_entropy(rho, eps=1e-12):
    vals = eigvalsh(rho)
    vals = np.clip(np.real(vals), eps, None)
    return -np.sum(vals * np.log2(vals))

def purity(rho):
    return np.real_if_close(np.trace(rho @ rho))

def trace_distance(rho, sigma):
    vals = eigvalsh(rho - sigma)
    return 0.5 * np.sum(np.abs(np.real(vals)))

def classical_fidelity_from_counts(countsA, countsB, shots=None):
    if shots is None:
        shots = max(sum(countsA.values()) if countsA else 1,
                    sum(countsB.values()) if countsB else 1)
    keys = sorted(set(countsA.keys()) | set(countsB.keys()))
    p = np.array([countsA.get(k, 0) / shots for k in keys])
    q = np.array([countsB.get(k, 0) / shots for k in keys])
    return np.sum(np.sqrt(p * q))

# -------- bloco de camada --------
def block_layer(circuit, qubits, seed=None):
    rng = random.Random(seed)
    n = len(qubits)
    for q in qubits:
        theta = rng.uniform(0, 2 * math.pi)
        phi = rng.uniform(0, 2 * math.pi)
        circuit.ry(theta, q)
        circuit.rz(phi, q)
    if n >= 2:
        for i in range(n-1):
            circuit.cx(qubits[i], qubits[i+1])
    return circuit

def build_circuit_with_depth(n_qubits, depth, seed=None):
    qr = QuantumRegister(n_qubits, 'q')
    qc = QuantumCircuit(qr)
    rng = random.Random(seed)
    for q in range(n_qubits):
        qc.h(q)
        qc.rz(rng.uniform(0, 2*math.pi), q)
    for d in range(depth):
        block_layer(qc, list(range(n_qubits)), seed=(seed, d))
    return qc

# -------- varredura de profundidade (sem Aer / sem transpile) --------
def depth_sweep_experiment(n_qubits=3, depths=range(1,11), seed=42, shots=2000):
    # estado de referência (depth = 1)
    ref_qc = build_circuit_with_depth(n_qubits, 1, seed=seed)
    ref_sv = Statevector.from_instruction(ref_qc)
    ref_dm = DensityMatrix(ref_sv).data

    results = {
        'depths': [],
        'fidelity': [],
        'entropy': [],
        'purity': [],
        'trace_distance': [],
        'classical_fidelity': [],
        'reference_dm': ref_dm,
    }

    for d in depths:
        qc = build_circuit_with_depth(n_qubits, d, seed=seed)
        sv = Statevector.from_instruction(qc)      # simula diretamente o estado
        dm = DensityMatrix(sv).data               # matriz densidade numpy

        fid = state_fidelity(DensityMatrix(dm), DensityMatrix(ref_dm))
        S = von_neumann_entropy(dm)
        P = purity(dm)
        TD = trace_distance(dm, ref_dm)

        # fidelidade clássica (distribuições na base computacional)
        probs = np.real(np.diag(dm))
        ref_probs = np.real(np.diag(ref_dm))
        dim = 2 ** n_qubits
        keys = [format(i, '0{}b'.format(n_qubits))[::-1] for i in range(dim)]
        counts = {k: int(round(p * shots)) for k, p in zip(keys, probs)}
        ref_counts = {k: int(round(p * shots)) for k, p in zip(keys, ref_probs)}
        class_fid = classical_fidelity_from_counts(counts, ref_counts, shots=shots)

        results['depths'].append(d)
        results['fidelity'].append(fid)
        results['entropy'].append(S)
        results['purity'].append(P)
        results['trace_distance'].append(TD)
        results['classical_fidelity'].append(class_fid)

        print(f"Depth {d:2d} → fidelity {fid:.6f}, S {S:.6f}, purity {P:.6f}, trace_dist {TD:.6f}, class_fid {class_fid:.6f}")

    return results
